In [5]:
import sys
sys.path.append('/cluster/sj1/bb_opt/src')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import pyro
import numpy as np
from scipy.stats import kendalltau
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from bb_opt.src.bayesian_opt import (
    optimize,
    get_model_bnn,
    train_model_bnn,
    partial_train_model_bnn,
    train,
    bnn_predict
)
from utils import get_path, save_pyro_model, load_pyro_model
import hsic
import knn_mi
from gpu_utils.utils import gpu_init
from tqdm import tnrange

gpu_id = gpu_init(best_gpu_metric="mem")
print(f"Running on GPU {gpu_id}")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%matplotlib inline

/cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Running on GPU 1


In [3]:
import tensorflow as tf
import keras
import os
from tensorflow.keras.backend import set_session


Using TensorFlow backend.


In [4]:
import h5py

In [6]:
zinc250k = []
with open('/cluster/sj1/bb_opt/chemical_vae/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv') as f:
    next(f)
    for line in f:
        line = [k.strip() for k in line.strip().split('\t')]
        zinc250k += [[line[0], [float(k) for k in line[1:]]]]

In [7]:
from chemvae_keras import vae_utils
from chemvae_keras import mol_utils as mu

In [8]:
is_cuda = True
directory = '../chemical_vae/models/zinc_properties'
n_init_samples = 1000
n_opt_samples = 10
cur_dir = '/cluster/sj1/bb_opt/src'
score_fn = lambda x : x[0]

In [9]:
os.chdir(cur_dir)
vae = vae_utils.VAEUtils(directory=directory)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


From /cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1208: calling reduce_prod (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
/cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/models.py:258: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Instructions for updating:
keep_dims is deprecated, use keepdims instead


From /cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1190: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


From /cluster/sj1/anaconda2/envs/py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1154: calling reduce_max (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Using standarized functions? True
Standarization: estimating mu and std values ...done!


In [10]:
zinc250k.sort(key=lambda k: score_fn(k[1]))

In [11]:
smiles = []
i = 0
for smiles_string, props in zinc250k[:2*n_init_samples]:
    i += 1
    if i % 10000 == 0:
        print("done {:d}K samples".format(i//1000))
    smiles += [vae.smiles_to_hot(mu.canon_smiles(smiles_string), canonize_smiles=True)]

In [ ]:
x_samples = [vae.encode(k)[0] for k in smiles[:n_init_samples]]
predictor_model = lambda x : score_fn(vae.predict_prop_Z(x[np.newaxis, :])[0])